In [1]:
import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, SimpleRNN,LSTM, Activation, Bidirectional
from keras.utils import np_utils

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
train = pd.read_csv('dataset/train_emoji.csv',header=None)
test = pd.read_csv('dataset/test_emoji.csv',header=None)

train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [3]:
import emoji


emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":grinning_face_with_big_eyes:",
                    "3": ":disappointed_face:",
                    "4": ":fork_and_knife:",
                    "5": ":hundred_points:",
                    "6": ":fire:",
                    "7": ":face_blowing_a_kiss:",
                    "8": ":chestnut:",
                    "9":":flexed_biceps:"
                   }


In [4]:
#emoji.EMOJI_UNICODE

In [5]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😃
😞
🍴
💯
🔥
😘
🌰
💪


In [6]:
data  = train.values
for i in range(5):
    print(data[i][0],emoji.emojize(emoji_dictionary[str(data[i][1])]))
    

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴


In [7]:
print(data.shape)

(132, 4)


In [8]:
X_train = train[0]
Y_train = train[1]

X_test = test[0]
Y_test = test[1]

print(X_test.shape)

(56,)


In [9]:

for ix in range(X_train.shape[0]):
    X_train[ix] = X_train[ix].split()
    
for ix in range(X_test.shape[0]):
    X_test[ix] = X_test[ix].split()

/home/ramendra/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ramendra/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
print(X_train.shape)

(132,)


In [11]:
Y_train = np_utils.to_categorical(Y_train)
        

In [12]:
Yt = np_utils.to_categorical(Y_test)

In [13]:
print(X_train[0],Y_train[0])

['never', 'talk', 'to', 'me', 'again'] [0. 0. 0. 1. 0.]


In [18]:
embeddings_index = {}
f = open('glove.6B.50d.txt',encoding='utf-8')

In [19]:
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float')
    embeddings_index[word] = coefs
f.close()

In [20]:
embeddings_index['food'].shape

(50,)

In [21]:
def getEmbeddingMatrix(sentences):
    embedding_matrix = np.zeros((X_train.shape[0],10,50))
    
    for ix in range(sentences.shape[0]):
        for ij in range(len(sentences[ix])):
            embedding_matrix[ix][ij] = embeddings_index[X_train[ix][ij].lower()]
    
    return embedding_matrix

In [22]:
embedding_matrix_train = np.zeros((X_train.shape[0], 10, 50))
embedding_matrix_test = np.zeros((X_test.shape[0], 10, 50))

for ix in range(X_train.shape[0]):
    for ij in range(len(X_train[ix])):
        embedding_matrix_train[ix][ij] = embeddings_index[X_train[ix][ij].lower()]
        
for ix in range(X_test.shape[0]):
    for ij in range(len(X_test[ix])):
        try:
            embedding_matrix_test[ix][ij] = embeddings_index[X_test[ix][ij].lower()]   
        except:
            embedding_matrix_test[ix][ij] = np.zeros((50,))

In [23]:
print(embedding_matrix_train.shape,embedding_matrix_test.shape)

(132, 10, 50) (56, 10, 50)


In [32]:
model = Sequential()

#model.add(LSTM(64, input_shape=(10,50), return_sequences=True,recurrent_dropout=0.1))
#model.add(Dropout(0.5))
model.add(LSTM(64,input_shape=(10,50), return_sequences=False,recurrent_dropout=0.1))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_2 (Activation)    (None, 5)                 0         
Total params: 29,765
Trainable params: 29,765
Non-trainable params: 0
_________________________________________________________________


In [33]:
from keras.callbacks import ModelCheckpoint # save the best model, fight overfiitting
from keras.callbacks import EarlyStopping #save time


checkpoint = ModelCheckpoint("best_model.h5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
earlystop = EarlyStopping(monitor='val_acc',patience=1)



earlystop = EarlyStopping(patience=10)

hist = model.fit(embedding_matrix_train,Y_train,
                epochs = 200, batch_size=32,shuffle=True,validation_split=0.1,callbacks=[checkpoint,earlystop])

Train on 118 samples, validate on 14 samples
Epoch 1/200
118/118 [==============================] - 1s 6ms/step - loss: 1.5964 - acc: 0.2373 - val_loss: 1.5815 - val_acc: 0.2857
Epoch 2/200
118/118 [==============================] - 0s 270us/step - loss: 1.5675 - acc: 0.3729 - val_loss: 1.5889 - val_acc: 0.2143
Epoch 3/200
118/118 [==============================] - 0s 352us/step - loss: 1.5429 - acc: 0.3729 - val_loss: 1.5919 - val_acc: 0.2143
Epoch 4/200
118/118 [==============================] - 0s 320us/step - loss: 1.5206 - acc: 0.3814 - val_loss: 1.5935 - val_acc: 0.2143
Epoch 5/200
118/118 [==============================] - 0s 364us/step - loss: 1.4877 - acc: 0.3983 - val_loss: 1.5948 - val_acc: 0.2857
Epoch 6/200
118/118 [==============================] - 0s 331us/step - loss: 1.4555 - acc: 0.3898 - val_loss: 1.5944 - val_acc: 0.2857
Epoch 7/200
118/118 [==============================] - 0s 391us/step - loss: 1.4262 - acc: 0.3814 - val_loss: 1.5846 - val_acc: 0.2857
Epoch 8/200


In [34]:
pred = model.predict_classes(embedding_matrix_test)

In [35]:
print(pred)

[4 3 2 0 2 2 3 2 4 2 1 2 0 0 1 3 2 2 3 2 0 0 4 0 3 3 2 0 4 2 0 1 0 2 0 1 2
 4 4 2 1 0 0 1 2 2 2 2 0 3 3 0 3 2 2 3]


In [36]:
t = np.array(test[1])

In [37]:
float(sum(pred==t))/embedding_matrix_test.shape[0]

0.6785714285714286

In [38]:
model.load_weights("best_model.h5")

In [39]:
pred = model.predict_classes(embedding_matrix_test)
t = np.array(test[1])
float(sum(pred==t))/embedding_matrix_test.shape[0]

0.6071428571428571

In [46]:
for e in range(pred.shape[0]):
    st = ''
    for ix in X_test[e]:
        st+=ix + ' '
    print(st)
    print(emoji.emojize(emoji_dictionary[str(pred[e])]),'\t',emoji.emojize(emoji_dictionary[str(t[e])]))

I want to eat 
🍴 	 🍴
he did not answer 
😞 	 😞
he got a raise 
😞 	 😃
she got me a present 
❤️ 	 ❤️
ha ha ha it was so funny 
😃 	 😃
he is a good friend 
😃 	 ❤️
I am upset 
😞 	 ❤️
We had such a lovely dinner tonight 
😃 	 ❤️
where is the food 
🍴 	 🍴
Stop making this joke ha ha ha 
😃 	 😃
where is the ball 
⚾ 	 ⚾
work is hard 
😃 	 😞
This girl is messing with me 
❤️ 	 😞
are you serious ha ha 
😞 	 😃
Let us go play baseball 
⚾ 	 ⚾
This stupid grader is not working 
😞 	 😞
work is horrible 
😞 	 😞
Congratulation for having a baby 
😃 	 😃
stop messing around 
😞 	 😞
any suggestions for dinner 
😃 	 🍴
I love taking breaks 
❤️ 	 ❤️
you brighten my day 
😞 	 😃
I boiled rice 
🍴 	 🍴
she is a bully 
❤️ 	 😞
Why are you feeling bad 
😞 	 😞
I am upset 
😞 	 😞
I worked during my birthday 
😃 	 😞
My grandmother is the love of my life 
❤️ 	 ❤️
enjoy your break 
😞 	 😃
valentine day is near 
😃 	 ❤️
I miss you so much 
❤️ 	 ❤️
throw the ball 
⚾ 	 ⚾
My life is so boring 
😞 	 😞
she said yes 
😞 	 😃
will you be my valentine